# prepare OLLAMA

In [1]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [9]:
import os
import asyncio

# NB: You may need to set these depending and get cuda working depending which backend you are running.
# Set environment variable for NVIDIA library
# Set environment variables for CUDA
os.environ['PATH'] += ':/usr/local/cuda/bin'
# Set LD_LIBRARY_PATH to include both /usr/lib64-nvidia and CUDA lib directories
os.environ['LD_LIBRARY_PATH'] = '/usr/lib64-nvidia:/usr/local/cuda/lib64'

async def run_process(cmd):
    print('>>> starting', *cmd)
    process = await asyncio.create_subprocess_exec(
        *cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE
    )

    # define an async pipe function
    async def pipe(lines):
        async for line in lines:
            print(line.decode().strip())

        await asyncio.gather(
            pipe(process.stdout),
            pipe(process.stderr),
        )

    # call it
    await asyncio.gather(pipe(process.stdout), pipe(process.stderr))

In [10]:
import asyncio
import threading

async def start_ollama_serve():
    await run_process(['ollama', 'serve'])

def run_async_in_thread(loop, coro):
    asyncio.set_event_loop(loop)
    loop.run_until_complete(coro)
    loop.close()

# Create a new event loop that will run in a new thread
new_loop = asyncio.new_event_loop()

# Start ollama serve in a separate thread so the cell won't block execution
thread = threading.Thread(target=run_async_in_thread, args=(new_loop, start_ollama_serve()))
thread.start()

>>> starting

In [11]:
!sleep 2

 ollama serve
time=2024-04-12T00:32:56.210Z level=INFO source=images.go:804 msg="total blobs: 6"
time=2024-04-12T00:32:56.212Z level=INFO source=images.go:811 msg="total unused blobs removed: 0"
time=2024-04-12T00:32:56.212Z level=INFO source=routes.go:1118 msg="Listening on 127.0.0.1:11434 (version 0.1.31)"
time=2024-04-12T00:32:56.246Z level=INFO source=payload_common.go:113 msg="Extracting dynamic libraries to /tmp/ollama316147731/runners ..."


In [12]:
# !ollama pull codellama
!ollama pull codellama:13b
# !ollama pull codellama:34b
# !ollama pull codellama:70b

time=2024-04-12T00:33:01.721Z level=INFO source=payload_common.go:140 msg="Dynamic LLM libraries [cpu cpu_avx cuda_v11 cpu_avx2 rocm_v60000]"
time=2024-04-12T00:33:01.721Z level=INFO source=gpu.go:115 msg="Detecting GPU type"
time=2024-04-12T00:33:01.721Z level=INFO source=gpu.go:265 msg="Searching for GPU management library libcudart.so*"
time=2024-04-12T00:33:01.815Z level=INFO source=gpu.go:311 msg="Discovered GPU libraries: [/tmp/ollama316147731/runners/cuda_v11/libcudart.so.11.0 /usr/local/cuda/lib64/libcudart.so.12.2.140]"
time=2024-04-12T00:33:01.877Z level=INFO source=gpu.go:120 msg="Nvidia GPU detected via cudart"
time=2024-04-12T00:33:01.877Z level=INFO source=cpu_common.go:11 msg="CPU has AVX2"
time=2024-04-12T00:33:02.100Z level=INFO source=gpu.go:188 msg="[cudart] CUDART CUDA Compute Capability detected: 7.5"
[GIN] 2024/04/12 - 00:33:02 | 200 |      63.268µs |       127.0.0.1 | HEAD     "/"
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling

# Prepare PandasAI

In [5]:
!pip install -q 'pandasai[langchain]'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.2/140.2 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 4.5/4.5 MB 106.4 MB/s eta 0:00:01time=2024-04-12T00:22:02.912Z level=INFO source=payload_common.go:140 msg="Dynamic LLM libraries [cpu_avx2 cuda_v11 rocm_v60000 cpu cpu_avx]"
time=2024-04-12T00:22:02.912Z level=INFO source=gpu.go:115 msg="Detecting GPU type"
time=2024-04-12T00:22:02.912Z level=INFO source=gpu.go:265 msg="Searching for GPU management library libcudart.so*"
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 4.5/4.5 MB 106.4 MB/s eta 0:00:01time=2024-04-12T00:22:02.993Z level=INFO source=gpu.go:311 msg="Discovered GPU libraries: [/tmp/ollama2795892889/runners/cuda_v11/libcudart.so.11.0 /usr/local/cuda/lib64/libcu

# Inference

In [22]:
import pandas as pd
from pandasai import SmartDataframe
from pandasai.llm.local_llm import LocalLLM

ollama_llm = LocalLLM(api_base="http://localhost:11434/v1", model="codellama:13b")
pre_url = ''

In [ ]:
#@title Inference (auto-run) { run: "auto" }
# @markdown ## step 1
# @markdown Choose conversation
OPENDATA_OPTIONS = '15-per-hour-seattle' #@param ['15-per-hour-seattle', 'american-assembly.bowling-green', 'brexit-consensus', 'canadian-electoral-reform', 'football-concussions', 'march-on.operation-marchin-orders', 'scoop-hivemind.affordable-housing', 'scoop-hivemind.biodiversity', 'scoop-hivemind.freshwater', 'scoop-hivemind.taxes', 'scoop-hivemind.ubi', 'ssis.land-bank-farmland.2rumnecbeh.2021-08-01', 'vtaiwan.uberx']

url =  ('https://raw.githubusercontent.com/'
    'compdemocracy/openData/master/'
    f'{OPENDATA_OPTIONS}/comments.csv')
if pre_url != url:
    df = SmartDataframe(pd.read_csv(url, index_col=0), config={"llm": ollama_llm})
    pre_url = url

# @markdown ##step 2
# @markdown Requst of analysis or request for chart
request = "Plot a hitogram about the distribution of agree, disagree and neutral comments against the topic" #@param
df.chat(request)